# Hosting MCP Server on Amazon Bedrock AgentCore Runtime with Strands Client

## Overview

In this tutorial we will learn how to host MCP (Model Context Protocol) servers on Amazon Bedrock AgentCore Runtime and connect them to local Strands agents. We will use the Amazon Bedrock AgentCore Python SDK to deploy MCP tools as a server compatible with Amazon Bedrock AgentCore.

The Amazon Bedrock AgentCore Python SDK handles the MCP server implementation details so you can focus on your tools' core functionality. It transforms your code into the AgentCore standardized MCP protocol contracts for direct communication.

### Tutorial Details

| Information         | Details                                                   |
|:--------------------|:----------------------------------------------------------|
| Tutorial type       | Hosting Tools with Agent Integration                      |
| Tool type           | MCP server with credit check functionality                |
| Tutorial components | Hosting MCP server on AgentCore Runtime + Strands Client |
| Tutorial vertical   | Mortgage/Financial Services                               |
| Example complexity  | Intermediate                                              |
| SDK used            | Amazon BedrockAgentCore Python SDK, MCP, and Strands     |

### Tutorial Architecture

In this tutorial we will describe how to deploy a credit check MCP server to AgentCore runtime and connect it to a local Strands agent for mortgage processing workflows.

```
Local Strands Agent → MCP Client → AgentCore Runtime → MCP Server (Credit Check)
```

### Tutorial Key Features

* Creating MCP servers with custom credit check tools
* Testing MCP servers locally and remotely
* Hosting MCP servers on Amazon Bedrock AgentCore Runtime
* Connecting Strands agents to remote MCP servers
* Invoking deployed MCP servers with authentication
* Building mortgage assistant workflows

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials configured
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) library
* Strands Agents SDK
* Docker running

## Step 1: Install Dependencies

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

* **Tools**: Functions that the AI can call to perform actions
* **Streamable HTTP**: Transport protocol used by AgentCore Runtime
* **Session Isolation**: Each client gets isolated sessions via `Mcp-Session-Id` header
* **Stateless Operation**: Servers must support stateless operation for scalability

AgentCore Runtime expects MCP servers to be hosted on `0.0.0.0:8000/mcp` as the default path.

## Step 2: Creating MCP Server for Credit Check

Now let's create our MCP server with credit check functionality. The server uses FastMCP with `stateless_http=True` which is required for AgentCore Runtime compatibility.

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def credit_check(customer_id: int) -> int:
    """Perform credit check for customer ID"""
    scores = {1111: 70, 2222: 80, 3333: 90}
    return scores.get(customer_id, 0)

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

In [ ]:
%%writefile requirements.txt
mcp>=1.10.0
boto3
bedrock-agentcore
bedrock-agentcore-starter-toolkit

### What This Code Does

* **FastMCP**: Creates an MCP server that can host your tools
* **@mcp.tool()**: Decorator that turns your Python functions into MCP tools
* **stateless_http=True**: Required for AgentCore Runtime compatibility
* **credit_check**: Custom tool for mortgage credit verification workflows

## Step 3: Setting up Amazon Cognito for Authentication

AgentCore Runtime requires authentication. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server.

In [ ]:
import sys
import os
from utils import setup_cognito_user_pool

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

%store cognito_config

## Step 4: Configuring AgentCore Runtime Deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code. The AgentCore Runtime provides:

* **Serverless Execution**: No infrastructure management required
* **Auto-scaling**: Handles traffic spikes automatically
* **Security**: Built-in authentication and authorization
* **Observability**: Integrated logging and monitoring

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_config['client_id']],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="credit_check_mcp_server"
)
print("Configuration completed ✓")

## Step 5: Launching MCP Server to AgentCore Runtime

Now that we've got a docker file, let's launch the MCP server to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

The launch process:
1. **Builds Docker Image**: Packages your MCP server code
2. **Pushes to ECR**: Stores the image in Amazon Elastic Container Registry
3. **Creates Runtime**: Deploys the serverless AgentCore Runtime
4. **Configures Authentication**: Sets up JWT-based access control

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## Step 6: Checking AgentCore Runtime Status

Now that we've deployed the AgentCore Runtime, let's check for its deployment status and wait for it to be ready. The runtime goes through several states:

* **CREATING**: Initial deployment in progress
* **READY**: Runtime is available and can serve requests
* **CREATE_FAILED**: Deployment encountered an error

In [ ]:
print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

## Step 7: Storing Configuration for Remote Access

Before we can invoke our deployed MCP server, let's store the Agent ARN and Cognito configuration in AWS Systems Manager Parameter Store and AWS Secrets Manager for easy retrieval by our Strands agent client.

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    secrets_client.create_secret(
        Name='credit_check_mcp/cognito/credentials',
        Description='Cognito credentials for credit check MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='credit_check_mcp/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

ssm_client.put_parameter(
    Name='/credit_check_mcp/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for credit check MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")
print(f"Agent ARN: {launch_result.agent_arn}")

## Step 8: Creating Strands Agent Client

Now let's create a Strands agent that connects to our deployed MCP server. Strands Agents provides:

* **Native MCP Support**: Built-in MCP client capabilities
* **Tool Integration**: Seamless integration of remote MCP tools
* **Model Agnostic**: Works with various LLM providers
* **Agent Orchestration**: Supports multi-agent workflows

The agent will retrieve credentials from AWS and connect to the remote MCP server using authenticated HTTP requests.

In [ ]:
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from datetime import timedelta

def create_mortgage_agent_with_mcp():
    """Create Strands agent connected to remote MCP server"""
    
    # Get credentials from AWS
    ssm_client = boto3.client('ssm', region_name=region)
    agent_arn_response = ssm_client.get_parameter(Name='/credit_check_mcp/runtime/agent_arn')
    agent_arn = agent_arn_response['Parameter']['Value']
    
    secrets_client = boto3.client('secretsmanager', region_name=region)
    response = secrets_client.get_secret_value(SecretId='credit_check_mcp/cognito/credentials')
    secret_value = response['SecretString']
    parsed_secret = json.loads(secret_value)
    bearer_token = parsed_secret['bearer_token']
    
    # Build MCP URL
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"🔗 Connecting to MCP server: {mcp_url}")
    
    # Create MCP client
    mcp_client = MCPClient(
        lambda: streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120))
    )
    
    return mcp_client, mcp_url

mcp_client, mcp_url = create_mortgage_agent_with_mcp()
print("✓ MCP client created")

%store mcp_url

## Step 9: Testing MCP Connection

Let's test the connection between our Strands agent and the remote MCP server. This validates:

* **Authentication**: JWT token is valid and accepted
* **Network Connectivity**: Agent can reach the AgentCore Runtime
* **MCP Protocol**: Tool discovery and registration works correctly
* **Integration**: Strands agent can use remote MCP tools

In [ ]:
try:
    with mcp_client:
        # Create Strands agent
        mortgage_agent = Agent(
            name="Mortgage Assistant",
            system_prompt="You help with mortgage applications. Use credit_check tool when needed."
        )
        
        # Get tools from MCP server
        tools = mcp_client.list_tools_sync()
        print(f"📋 Available tools: {[t.tool_name for t in tools]}")
        
        # Add tools to agent
        mortgage_agent.tool_registry.process_tools(tools)
        
        print("✅ Successfully connected to remote MCP server!")
        
except Exception as e:
    print(f"❌ Connection failed: {e}")

## Step 10: Testing Credit Check Functionality

Now let's test our mortgage assistant with real credit check queries. This demonstrates the end-to-end workflow:

1. **User Query**: Natural language request for credit information
2. **Agent Processing**: Strands agent interprets the request
3. **Tool Invocation**: Agent calls the remote credit_check MCP tool
4. **AgentCore Execution**: Runtime processes the MCP request
5. **Response**: Credit score returned through the MCP protocol
6. **Agent Response**: Natural language response to the user

In [ ]:
# Test queries
test_queries = [
    "What is the credit score for customer 1111?",
    "Check credit for customer 2222",
    "Verify creditworthiness of customer 3333"
]

with mcp_client:
    for query in test_queries:
        print(f"\n🎯 Query: {query}")
        try:
            response = mortgage_agent(query)
            print(f"💬 Response: {response.message}")
        except Exception as e:
            print(f"❌ Error: {e}")

## Benefits of This Architecture

✅ **Scalability**: AgentCore automatically scales MCP servers based on demand  
✅ **Security**: Built-in AWS authentication, authorization, and network isolation  
✅ **Reliability**: Enterprise-grade hosting with high availability and fault tolerance  
✅ **Flexibility**: Local agents can connect to remote services across regions  
✅ **Standards**: MCP protocol ensures consistent tool interfaces and interoperability  
✅ **Cost Efficiency**: Pay-per-use serverless model with no idle costs  
✅ **Observability**: Built-in logging, monitoring, and tracing capabilities  

## Next Steps

Now that you have successfully deployed an MCP server to AgentCore Runtime and connected it to a Strands agent, you can:

1. **Add More Tools**: Extend your MCP server with additional mortgage-related tools
2. **Multi-Agent Systems**: Create specialized agents for different mortgage workflows
3. **Custom Authentication**: Implement custom JWT authorizers for enterprise integration
4. **Integration**: Integrate with other AgentCore services like Memory and Gateway
5. **Production Deployment**: Scale to production with proper CI/CD pipelines

## Cleanup (Optional)

If you want to clean up the resources created during this tutorial, run the following cells:

In [ ]:
print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
cognito_client = boto3.client('cognito-idp', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = launch_result.ecr_uri.split('/')[1]
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    try:
        ssm_client.delete_parameter(Name='/credit_check_mcp/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")
        
    try:
        secrets_client.delete_secret(
            SecretId='credit_check_mcp/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    # Delete Cognito User Pool
    try:
        cognito_client.delete_user_pool(UserPoolId=cognito_config['user_pool_id'])
        print("✓ Cognito User Pool deleted")
    except Exception as e:
        print(f"⚠ Error deleting Cognito User Pool: {e}")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

## Benefits

✅ **Scalability**: AgentCore auto-scales MCP servers  
✅ **Security**: Built-in AWS authentication and authorization  
✅ **Reliability**: Enterprise-grade hosting with high availability  
✅ **Flexibility**: Local agents can connect to remote services  
✅ **Standards**: MCP protocol ensures consistent interfaces  

## Summary

You have successfully:
- Created a credit check MCP server
- Deployed it to Bedrock AgentCore Runtime
- Connected a local Strands agent to the remote MCP server
- Tested the integration with real credit check queries

The architecture demonstrates how to scale from local development to production-ready, cloud-hosted MCP services while maintaining the same client interface.